In [ ]:
import src.dataloader
import src.loss
import src.transforms as t
import src.functional
from src.models.HCNet import HCNet
from src.models.RDCNet import RDCNet
from src.models.unet import Unet_Constructor as unet

import torch
import torch.nn
from torch.utils.data import DataLoader

import time
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms
from torch.utils.tensorboard import SummaryWriter
from scipy.ndimage.morphology import binary_fill_holes, binary_erosion, binary_dilation
import skimage.io as io
from importlib import reload  

In [ ]:
reload(src.loss)

In [ ]:
model = torch.jit.script(HCNet(in_channels=3, out_channels=4, complexity=15)).cuda()
#model = RDCNet(in_channels=3, out_channels=4, complexity=15).train().cuda()

#model.load_state_dict(torch.load('val_Dec30_1.hcnet2'))
model.load_state_dict(torch.load('Jan15_1_background_reject.hcnet'))
#model.load_state_dict(torch.load('Jan11_1.hcnet'))
model.train()

model = unet(image_dimensions=3,
                 in_channels=3,
                 out_channels=4,
                 feature_sizes=[8,16],
                 kernel=(3, 3, 2),
                 upsample_kernel=(3, 3, 2),
                 max_pool_kernel=(2, 2, 2),
                 upsample_stride=2,
                 dilation=1,
                 groups=1,
                 ).cuda().train()

In [ ]:
reload(src.transforms)
print('Loading Train...')
transforms = torchvision.transforms.Compose([
    t.random_affine(shear=(-15, 15), scale=(0.75, 1.5)),
    t.nul_crop(rate=.65),
    t.random_crop(shape=(256, 256, 16)),
    t.elastic_deformation(grid_shape=(3, 3, 2), scale=1.2),
    t.to_cuda(),    
    t.random_h_flip(),
    t.random_v_flip(),
    t.gaussian_blur(rate=0.25),
    t.adjust_brightness(range_brightness = (-0.05, 0.15)),
    t.random_noise(gamma=0.2),
])

data = src.dataloader.dataset('./data/train', transforms=transforms, random_ind=True)
dl = DataLoader(data, batch_size=1, shuffle=False, num_workers=0)
print('Done')



In [ ]:
transforms = torchvision.transforms.Compose([
    t.random_affine(shear=(-15, 15), scale=(0.75, 1.5)),
    t.random_crop(shape=(256, 256, 16)),
    t.elastic_deformation(grid_shape=(3, 3, 2), scale=1.2),
    t.to_cuda(),    
    t.random_h_flip(),
    t.random_v_flip(),
    t.gaussian_blur(rate=0.25),
    t.adjust_brightness(range_brightness = (-0.05, 0.15)),
    t.random_noise(gamma=0.2),
])
nul_data = src.dataloader.nul_dataset('./data/background', transforms=transforms, random_ind=True)
nd = DataLoader(data, batch_size=1, shuffle=False, num_workers=0)

If the sigma is too low, basicall the network cannot influence distance cells. I.e. vectors can point any way they want, and because they dont have a center near them, they wont affect training. This in theory should be compensated by the prob map, but idk...

In [ ]:
print('Loading Val...')
val_trans = torchvision.transforms.Compose([
    t.nul_crop(1),
    t.random_crop(shape=(256, 256, 26)),
    t.to_cuda(),    
])
val = src.dataloader.dataset('/media/DataStorage/Dropbox (Partners HealthCare)/HairCellInstance/data/validate', transforms=val_trans)
val = DataLoader(val, batch_size=1, shuffle=False, num_workers=0)
print('Done')

In [ ]:
lr = 1e-4
gamma = 1#0.993
wd = .001
#sigma= 0.1 
sigma = lambda e: torch.tensor([0.002]) #+ 0.02 * .99**e])
iterations=5

In [ ]:
sigma = lambda e: torch.tensor([0.002]) #+ 0.02 * .99**e])

In [ ]:
try:
    if e > 0:
        pass
except:
    epochs = 500
    e = -1

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay = wd)
loss_fun = src.loss.tversky_loss()
nul_loss_fun = src.loss.l1_loss()

In [ ]:
optimizer.weight_decay = wd
optimizer.lr = lr
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 300], gamma=0.1)

In [ ]:
try:
    save_file = 'Feb15_1.hcnet'
    writer
    
except:
    writer = SummaryWriter()
    best_train_loss = 2
    best_val_loss = 2


In [ ]:
model.train()
print('')

PROBLEM: model sees sgn's and thinks its cells


SOLUTION:
give it random SGN z_stacks with no cells in it. Let the model guess centroids with src.functional.estimate_centroids. Create zero matrix of same size. Run backprop against this. See if it works. 

In [ ]:
#sigma = torch.tensor([0.01]).cuda()  
while True:
    e += 1
    epoch_loss = []
    for data_dict in dl:
        image = data_dict['image']
        image = (image - 0.5) / 0.5
        mask = data_dict['masks'] > 0.5
        centroids = data_dict['centroids']

        if centroids.shape[1] == 0:
            continue

        optimizer.zero_grad()

        out = model(image.cuda())#, iterations)
           
            
        prob_map = torch.sigmoid(out[:, -1, ...]).unsqueeze(1)
        out = out[:, 0:3:1, ...]
        out = src.functional.vector_to_embedding(out)
        out = src.functional.embedding_to_probability(out, centroids.cuda(), sigma(e).cuda())
        loss = 3*loss_fun(out, mask.cuda(),alpha=0.5, beta=0.5) \
               + loss_fun(prob_map, mask.cuda().sum(1).unsqueeze(1), alpha=1, beta=1)

        if torch.any(torch.isnan(out)):
            continue
            
        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.detach().cpu().item())
    
#     nul_loss = []
#     for nul_dict in nd: # This is the dataloader specically for nul datasets
#         """
#         Nul dict training step
        
#         Predict an outcome of an image without any hair cells in the image
#         Guess centroids - they'll all be wrong if any
#         use loss function to do backwards pass - train network to ignore any of these random cells hopefully
#         """
#         image = nul_dict['image']
#         image = (image - 0.5) / 0.5
#         mask = nul_dict['masks'] > 0.5
        
#         optimizer.zero_grad()
        
#         # Predict an output - if there is anything it will be wrong
#         out = model(image.cuda())#,iterations)
#         prob_map = torch.sigmoid(out[:, -1, ...]).unsqueeze(1)
#         out = out[:, 0:3:1, ...]
#         embed = src.functional.vector_to_embedding(out)
        
#         try:
#             centroids = src.functional.estimate_centroids(embed, 1, 50)
#         except ValueError: 
#             continue
            
#         del embed
        
#         if centroids.shape[0] == 0:
#             continue
            
#         out = src.functional.vector_to_embedding(out)
#         out = src.functional.embedding_to_probability(out, centroids.cuda(), sigma(e).cuda())
        
#         # Loss function needs to handle no positive labels
#         loss = nul_loss_fun(out, torch.zeros(out.shape).cuda()) \
#         + nul_loss_fun(prob_map, torch.zeros(prob_map.shape).cuda())
        
#         nul_loss.append(float(out.shape[1]))
#         loss.backward()
#         optimizer.step()
                     
#     writer.add_scalar('Loss/nul', torch.mean(torch.tensor(nul_loss)).item(), e)
    writer.add_scalar('Loss/train', torch.mean(torch.tensor(epoch_loss)).item(), e)
    writer.add_scalar('AdjustedLoss/train', torch.mean(torch.tensor(epoch_loss))*sigma(e), e)
    writer.add_scalar('Hyperparam/lr',scheduler.get_lr()[0],e)
    writer.add_scalar('Hyperparam/weight_decay',wd,e)
    writer.add_scalar('Hyperparam/sigma', sigma(e).item(),e)

    scheduler.step()
    data.step()
    nul_data.step()
        
    if e % 10 == 0:
        del data_dict
        
        with torch.no_grad():
            val_loss = []
            for val_dict in val:
                image = val_dict['image'].sub(0.5).div(0.5)
                mask = val_dict['masks'] > 0.5
                centroids = val_dict['centroids']

                out = model(image.cuda())#, iterations)

                prob_map = torch.sigmoid(out[:, -1, ...]).unsqueeze(1)
                out = out[:, 0:3:1, ...]
                out = src.functional.vector_to_embedding(out)
                out = src.functional.embedding_to_probability(out, centroids.cuda(), sigma(e).cuda())
                loss = 3*loss_fun(out, mask.cuda(),alpha=0.5, beta=0.5) \
                       + loss_fun(prob_map, mask.cuda().sum(1).unsqueeze(1), alpha=1, beta=1)

                val_loss.append(loss.item())
                
        val_loss = torch.mean(torch.tensor(val_loss)).item()

        writer.add_scalar('Loss/validate', val_loss, e)


        torch.save(model.state_dict(), 'train_' + save_file)


In [ ]:
plt.imshow(data_dict['masks'][0,:,:,:,:].sum(0).sum(-1).cpu())
plt.plot(data_dict['centroids'][0,:,1].cpu(), data_dict['centroids'][0,:,0].cpu(), 'bo')
plt.show()

In [ ]:
image.max()

In [ ]:
save_name = 'Jan15_2.hcnet'
torch.save(model.state_dict(), save_name)

In [ ]:
model.train()
s = torch.tensor([0.002]).cuda()  
with torch.no_grad():
    for data_dict in val:
        image = data_dict['image']
        image = (image - 0.5) / 0.5
        mask = data_dict['masks'] > 0.5
        
        centroids = data_dict['centroids']
        plt.imshow(data_dict['masks'][0,:,:,:,:].sum(0).sum(-1).cpu())
        plt.plot(data_dict['centroids'][0,:,1].cpu(), data_dict['centroids'][0,:,0].cpu(), 'bo')
        plt.show()
        
        out = model(image.cuda(),iterations)
        prob_map = torch.sigmoid(out[:, -1, ...]).unsqueeze(1)
        out = out[:, 0:3:1, ...]
        out = src.functional.vector_to_embedding(out)
        out = src.functional.embedding_to_probability(out, centroids.cuda(), s)
        loss = 3*loss_fun(out, mask.cuda(),alpha=0.5, beta=0.5) \
               + loss_fun(prob_map, mask.cuda().sum(1).unsqueeze(1), alpha=1, beta=1)

        plt.imshow(data_dict['masks'][0,:,:,:,:].sum(0).sum(-1).cpu())
        plt.plot(data_dict['centroids'][0,:,1].cpu(), data_dict['centroids'][0,:,0].cpu(), 'bo')
        plt.show()
        
        break
        
model.train()
print(' ')
test = out.clone()

print(loss)

value, ind = out.max(1)
ind[prob_map[:,0,...] < 0.5]=0
#for i in range(render.shape[0]):
#    render[i,:,:,:] = binary_dilation(binary_fill_holes(render[i,:,:,:]))
ind = ind.cpu().detach().squeeze(0).float().numpy().transpose((2,1,0))
io.imsave('bigtest.tif', ind)
ind.min()


for data_dict in val:
    plt.imshow(data_dict['masks'][0,:,:,:,:].sum(0).sum(-1).cpu())
    plt.plot(data_dict['centroids'][0,:,1].cpu(), data_dict['centroids'][0,:,0].cpu(), 'bo')
    plt.show()
    

In [ ]:
del image, out, prob_map,ind, value, embed

Naieve Test

In [ ]:
s = torch.tensor([0.016]).cuda()  
model.train()
for m in model.modules():
    if isinstance(m, torch.nn.BatchNorm3d):
        m.eval()
with torch.no_grad():
    image = io.imread('/media/DataStorage/Dropbox (Partners HealthCare)/'+\
                      'HairCellInstance/data/augment_train/Oct 25 AAV2-PHP.B-CMV Olga L17 m3 G200 La0.25.lif - TileScan 1.tif')
    image = torch.from_numpy(image[..., [0, 2, 3]] / 2 ** 16).unsqueeze(0).float()
    image = image.transpose(1, 3).transpose(0, -1).squeeze().unsqueeze(0).sub(0.5).div(0.5).cuda()


    out = model(image.cuda())#, 5)
    print(image.shape, out.shape)
    prob_map = torch.sigmoid(out[:, -1, ...]).unsqueeze(1)

    out = out[:, 0:3:1, ...]
    embed = src.functional.vector_to_embedding(out)
    x = embed.detach().cpu().numpy()[0,0,...].flatten()
    y = embed.detach().cpu().numpy()[0,1,...].flatten()
    plt.figure(figsize=(15,15))
    plt.hist2d(x,y,bins=(image.shape[2], image.shape[3]), range=((0,1), (0,0.5)))
    plt.show()
    
    for i in range(3):
        embed[:,i,...][prob_map.squeeze(0) < 0.5] = -10
    
    centroids = src.functional.estimate_centroids(embed, 2.5, 150)
    embed = src.functional.vector_to_embedding(out)
    out = src.functional.embedding_to_probability(embed, centroids.cuda(), s)



model.train()
value, ind = out.max(1)
#ind[prob_map[:,0,...] < 0.5]=0

x = embed.detach().cpu().numpy()[0,0,...].flatten()
y = embed.detach().cpu().numpy()[0,1,...].flatten()
plt.figure(figsize=(15,15))
plt.hist2d(x,y,bins=(image.shape[2], image.shape[3]), range=((0,1), (0,0.5)))
plt.plot(centroids[0,:,0].detach().cpu().numpy(), centroids[0,:,1].detach().cpu().numpy(), 'ro')
# plt.plot(data_dict['centroids'][0,:,0].cpu(), data_dict['centroids'][0,:,1].cpu(), 'bo', alpha=0.5)
plt.show()

print(image.shape)
plt.imshow(prob_map[0,0,:,:,13].gt(0.95).cpu().numpy())
plt.show()

print(centroids.shape[1], ' cells predicted')
print(prob_map.max())

value, ind = out.max(1)
ind[value < 0.25] = 0
ind[prob_map.squeeze(1)<0.5]=0
correction_matrix = binary_dilation(binary_erosion(ind.cpu().numpy()))
ind[np.logical_not(binary_dilation)] = 0
io.imsave('naieve_test.tif', ind.cpu().detach().squeeze(0).float().numpy().transpose((2,1,0)))
ind.min()
ind.shape
del image, out, embed, centroids


In [ ]:
prob_map = prob_map > 0.95

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(prob_map.squeeze(0).squeeze(0).sum(-1).cpu().numpy())
plt.show()